<a href="https://colab.research.google.com/github/mehjabeenalam-ux/Learning_Python-for-Beginners/blob/main/I_O_Bound_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I/O-Bound Tasks

Spend most time waiting for external resources (network, disk, database, APIs, timers).
CPU is mostly idle during waits.
Best tools: asyncio (preferred modern choice), or threading.

This code downloads the same webpage 20 times in a row, one after another.
It measures and prints the size of each downloaded page in bytes, then shows the total time taken.

It runs completely synchronously → each request must fully finish before the next one starts, making it slow for many requests.

1. Sequential (slow baseline)

In [1]:
import time
import requests

urls = ["https://example.com"] * 20

start = time.time()
for url in urls:
    response = requests.get(url)
    print(len(response.content))
print(f"Total: {time.time() - start:.2f} s")

513
513
513
513
513
513
513
513
513
513
513
513
513
513
513
513
513
513
513
513
Total: 1.24 s


This version is the easiest to read and understand, which is why beginners (and many production scripts that only need 3–10 requests) still use it every day.

If the number of requests grows to dozens or hundreds → almost everyone switches to the asyncio version which is given below.

2. Using asyncio (recommended for I/O-bound)

In [4]:
import asyncio
import aiohttp
import time
import nest_asyncio

nest_asyncio.apply()

async def fetch(url, session):
    async with session.get(url) as response:
        return await response.read()

async def main():
    urls = ["https://example.com"] * 50
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(url, session) for url in urls]
        results = await asyncio.gather(*tasks)
        print([len(r) for r in results])

start = time.time()
asyncio.run(main())
print(f"Total: {time.time() - start:.2f} s")

[513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513]
Total: 0.37 s


It asks the same website (“https://example.com”) 20 times in a row, downloads the page content each time, prints how many bytes it received, and finally shows how long the whole process took.

This code downloads the same webpage 50 times, but in a very smart and fast way.

Here’s what happens in simple words:

- It makes a list of 50 copies of the same website address.
- It asks the computer to start getting all 50 pages at almost the same time (not one after another).
- While waiting for the internet to send each page, the computer works on the other requests.
- When all pages arrive, it shows how big each one is.
The whole thing finishes very quickly — often in just a few seconds.

- Normal slow way = one page at a time → takes a long time.

- This smart way = many pages together → finishes much faster. 😊

3. Using threading (still very good)

In [5]:
import time
import requests
from concurrent.futures import ThreadPoolExecutor

def fetch(url):
    response = requests.get(url)
    return len(response.content)

urls = ["https://example.com"] * 50

start = time.time()
with ThreadPoolExecutor(max_workers=20) as executor:
    results = list(executor.map(fetch, urls))
print(f"Total: {time.time() - start:.2f} s")

Total: 0.39 s


This code downloads the same webpage 50 times in parallel using multiple threads.

It creates a pool of 20 worker threads (using ThreadPoolExecutor)
It sends up to 20 download requests at the same time
Each thread runs the fetch() function → makes one HTTP request with requests.get()
executor.map() automatically gives each URL to a free thread
When all 50 requests finish, it shows the total time taken

Result: Much faster than doing one request after another
(usually 3–8 seconds instead of 20–60 seconds), because waiting time for the internet overlaps between the threads.